<a href="https://colab.research.google.com/github/thimnahindira/desafioengenheirodedados/blob/main/Desafio_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Spark Serasa
O objetivo deste desafio é entender seu domínio na linguagem Python, sua capacidade de criar Pipelines de dados e extrair informações relavantes de dados.


## Dados trabalhados

Este conjunto de dados foi generosamente fornecido pela maior loja de departamentos do mercado brasileiro. Depois que um cliente compra o produto, um vendedor é notificado para atender a esse pedido. Assim que o cliente recebe o produto, ou a data prevista de entrega está prevista, o cliente recebe um inquérito de satisfação por e-mail onde pode dar nota da experiência de compra e deixar alguns comentários. 

### Atenção
1.     Um pedido pode ter vários itens.
2.     Cada item pode ser atendido por um vendedor distinto.
3.     Todo o texto identificando lojas e parceiros foi substituído pelos nomes das grandes casas de Game of Thrones.

### Esquema de dados

Os dados são divididos em vários conjuntos de dados para melhor compreensão e organização. Consulte o esquema de dados a seguir ao trabalhar com ele:

![Data Schema](https://i.imgur.com/HRhd2Y0.png)

## Configuração do Spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install pyspark==3.3.1
!pip install -q findspark

tar: spark-3.3.1-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 28.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=c469fe181f4ed61c0de176e5c3181567e82c6c7e34b43f3d127f6332260f631b
  Stored in directory: /root/.cache/pip/wheels/51/c8/18/298a4ced8ebb3ab8a7d26a7198c0cc7035abb906bde94a4c4b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [1]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install pyspark==2.4.7
!pip install -q findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import findspark
#findspark.init()

In [3]:
findspark.init()

In [4]:
from pyspark.sql import SparkSession

### Criando Sessão do Spark

In [5]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Iniciando com Spark') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [6]:
spark

## Download do arquivo

In [7]:
!wget -c --no-check-certificate "https://onedrive.live.com/download?cid=08A3A632FA9EDBB1&resid=8A3A632FA9EDBB1%2110217&authkey=ACij76UvvMhutw8" -O dataset.zip

--2023-03-18 16:26:49--  https://onedrive.live.com/download?cid=08A3A632FA9EDBB1&resid=8A3A632FA9EDBB1%2110217&authkey=ACij76UvvMhutw8
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://pepxtw.dm.files.1drv.com/y4mZ7fZZjGhDso612n5_8PnybPWJsSBDyNLBLpI5MmSq2OQ4i_envOoCXFN0vsBVnzxtLZTYZkhl8KB6Hej-0-cOZeynhlGYllZyEUgnDFMOtA2fLhIsDdEM-ccx0KN37mME_XLGB9rzVScIGY3uIbEDEre8B6E3Glna36dL6qMhBCVJgdyED1OSeE_Rv-7JkqJGxQ_j6TDpQ_wasXAzBtSHw/brazilian-ecommerce.zip?download&psid=1 [following]
--2023-03-18 16:26:49--  https://pepxtw.dm.files.1drv.com/y4mZ7fZZjGhDso612n5_8PnybPWJsSBDyNLBLpI5MmSq2OQ4i_envOoCXFN0vsBVnzxtLZTYZkhl8KB6Hej-0-cOZeynhlGYllZyEUgnDFMOtA2fLhIsDdEM-ccx0KN37mME_XLGB9rzVScIGY3uIbEDEre8B6E3Glna36dL6qMhBCVJgdyED1OSeE_Rv-7JkqJGxQ_j6TDpQ_wasXAzBtSHw/brazilian-ecommerce.zip?download&psid=1
Resolving pepxtw.dm.files.1drv.com (pe

### Salvando CSVs
Os arquivos estaram disponíveis na pasta /content/dataset

In [8]:
!unzip /content/dataset.zip -d /content/dataset

Archive:  /content/dataset.zip
  inflating: /content/dataset/olist_customers_dataset.csv  
  inflating: /content/dataset/olist_geolocation_dataset.csv  
  inflating: /content/dataset/olist_order_items_dataset.csv  
  inflating: /content/dataset/olist_order_payments_dataset.csv  
  inflating: /content/dataset/olist_order_reviews_dataset.csv  
  inflating: /content/dataset/olist_orders_dataset.csv  
  inflating: /content/dataset/olist_products_dataset.csv  
  inflating: /content/dataset/olist_sellers_dataset.csv  
  inflating: /content/dataset/product_category_name_translation.csv  


# Desafios
Para realizar os desafios, use Python com Pyspark ou Spark SQL neste notebook, na célula de código abaixo do descritivo.

### 1 - Ingestão de dados
Salve os arquivos CSVs em um formato parquet, se atentando ao tipo correto de dados (salvar datas com o tipo adequado, números com o tipo correto, etc)

In [31]:
#Lendo os arquivos CSV no DataFrame

geolocation = spark.read.csv('/content/dataset/olist_geolocation_dataset.csv', sep=',', inferSchema=True, header=True)

customers = spark.read.csv('/content/dataset/olist_customers_dataset.csv', sep=',', inferSchema=True, header=True)

order_items = spark.read.csv('/content/dataset/olist_order_items_dataset.csv', sep=',', inferSchema=True, header=True)

order_payments = spark.read.csv('/content/dataset/olist_order_payments_dataset.csv', sep=',', inferSchema=True, header=True)

order_reviews = spark.read.csv('/content/dataset/olist_order_reviews_dataset.csv', sep=',', inferSchema=True, header=True)

orders = spark.read.csv('/content/dataset/olist_orders_dataset.csv', sep=',', inferSchema=True, header=True)

products = spark.read.csv('/content/dataset/olist_products_dataset.csv', sep=',',  inferSchema=True, header=True)

sellers = spark.read.csv('/content/dataset/olist_sellers_dataset.csv', sep=',', inferSchema=True, header=True)

product_category = spark.read.csv('/content/dataset/product_category_name_translation.csv', sep=',', inferSchema=True, header=True)

In [36]:
#Salvando arquivos em parquet

geolocation.write.parquet("geolocation.parquet")

customers.write.parquet("customers.parquet")

order_items.write.parquet("order_items.parquet")

order_payments.write.parquet("order_payments.parquet")

order_reviews.write.parquet("order_reviews.parquet")

orders.write.parquet("ordersn.parquet")

products.write.parquet("products.parquet")

sellers.write.parquet("sellers.parquet")

product_category.write.parquet("product_category.parquet")

## 2 - Crie uma base unificada
Crie uma base em parquet unificadando os datasets:
- olist_orders_dataset
- olist_order_items_dataset
- olist_order_payments_dataset
- olist_order_reviews_dataset

In [83]:
#Unificando bases pelo campo order_id
orders_dataset = order_items.join(orders, ["order_id"]) \
                            .join(order_payments,["order_id"]) \
                            .join(order_reviews, ["order_id"]) \
                            #

In [86]:
orders_dataset.write.parquet("orders_dataset.parquet")

## 3 - Gerar um arquivo Json
Salvar o olist_order_reviews_dataset como um único arquivo Json

In [87]:
## Gerando arquivo Json
order_reviews.write.json("order_reviews.json")

## 4 - Qual foi o faturamento mês a mês? Qual o mês com maior número de pedidos?

In [96]:
orders_dataset.dtypes

[('order_id', 'string'),
 ('order_item_id', 'int'),
 ('product_id', 'string'),
 ('seller_id', 'string'),
 ('shipping_limit_date', 'string'),
 ('price', 'double'),
 ('freight_value', 'double'),
 ('customer_id', 'string'),
 ('order_status', 'string'),
 ('order_purchase_timestamp', 'string'),
 ('order_approved_at', 'string'),
 ('order_delivered_carrier_date', 'string'),
 ('order_delivered_customer_date', 'string'),
 ('order_estimated_delivery_date', 'string'),
 ('payment_sequential', 'int'),
 ('payment_type', 'string'),
 ('payment_installments', 'int'),
 ('payment_value', 'double'),
 ('review_id', 'string'),
 ('review_score', 'string'),
 ('review_comment_title', 'string'),
 ('review_comment_message', 'string'),
 ('review_creation_date', 'string'),
 ('review_answer_timestamp', 'string')]

In [92]:
orders_dataset.groupBy("order_approved_at").sum("payment_value").show(truncate=False)

+-------------------+------------------+
|order_approved_at  |sum(payment_value)|
+-------------------+------------------+
|2017-11-22 02:49:29|322.6             |
|2017-11-24 01:49:32|79.2              |
|2018-06-19 13:57:28|449.68            |
|2017-11-19 13:35:31|91.66             |
|2018-05-05 09:55:20|35.79             |
|2018-08-11 19:05:20|163.68            |
|2017-03-25 20:10:17|321.71            |
|2017-08-18 00:44:22|48.69             |
|2017-09-18 21:55:12|283.33            |
|2017-10-03 10:14:14|45.93             |
|2017-03-24 10:35:35|52.77             |
|2018-01-04 15:47:21|268.55            |
|2018-01-05 23:47:32|76.08             |
|2017-05-26 02:35:30|194.14            |
|2018-01-16 09:54:26|82.98             |
|2017-03-21 15:22:48|61.26             |
|2017-08-28 21:24:26|74.16             |
|2018-06-28 09:51:03|283.56            |
|2018-07-10 04:25:25|90.83             |
|2018-05-15 13:55:27|199.8             |
+-------------------+------------------+
only showing top

In [88]:
## Code

df = order_payments.toPandas()
df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [90]:
df = orders.toPandas()
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [89]:
df = orders_dataset.toPandas()
df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,...,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,014405982914c2cde2796ddcf0b8703d,1,6782d593f63105318f46bbf7633279bf,325f3178fb58e2a9778334621eecdbf9,2017-08-01 17:50:17,27.90,3.81,2de342d6e5905a5a8bb3a991c855f3e2,delivered,2017-07-26 17:38:47,...,1,credit_card,7,78.43,ba5e6d78da2b2c3f5127ab99abe93629,5,None,None,2017-08-01 00:00:00,2017-08-04 19:34:54
1,014405982914c2cde2796ddcf0b8703d,2,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-08-01 17:50:17,21.33,25.39,2de342d6e5905a5a8bb3a991c855f3e2,delivered,2017-07-26 17:38:47,...,1,credit_card,7,78.43,ba5e6d78da2b2c3f5127ab99abe93629,5,None,None,2017-08-01 00:00:00,2017-08-04 19:34:54
2,019886de8f385a39b75bedbb726fd4ef,1,e9a69340883a438c3f91739d14d3a56d,1b4c3a6f53068f0b6944d2d005c9fc89,2018-02-15 13:08:12,159.90,28.50,8cf88d7ba142365ef2ca619ef06f9a0f,delivered,2018-02-10 12:52:51,...,1,credit_card,2,188.40,3a6ebeb7f45583720372e838e99cec6a,5,None,Produto foi entregue antes do prazo e em perfe...,2018-02-24 00:00:00,2018-02-27 15:10:57
3,01a6ad782455876aa89081449d49c452,1,036734b5a58d5d4f46b0616ddc047ced,ea8482cd71df3c1969d7b9473ff13abc,2018-01-24 10:17:29,34.99,15.10,71accffbcbdf8e02f67a469f65cdbf73,delivered,2018-01-18 10:07:52,...,1,credit_card,5,50.09,34fdf58aa22a5d9c4929b2b719a1b7f5,3,None,None,2018-02-02 00:00:00,2018-02-05 09:59:55
4,01d907b3e209269e120a365fc2b97524,1,b1434a8f79cb3528540d9b21e686e823,d1c281d3ae149232351cd8c8cc885f0d,2017-08-16 10:25:08,151.99,17.77,d02cc92f5e33eb58d9ff4d5cce6ae901,delivered,2017-08-09 16:21:06,...,1,credit_card,10,169.76,d551da337adf573d0a25ebfd02e9fb5c,5,None,None,2017-08-17 00:00:00,2017-08-20 20:01:58


In [93]:
orders_dataset.dtypes

[('order_id', 'string'),
 ('order_item_id', 'int'),
 ('product_id', 'string'),
 ('seller_id', 'string'),
 ('shipping_limit_date', 'string'),
 ('price', 'double'),
 ('freight_value', 'double'),
 ('customer_id', 'string'),
 ('order_status', 'string'),
 ('order_purchase_timestamp', 'string'),
 ('order_approved_at', 'string'),
 ('order_delivered_carrier_date', 'string'),
 ('order_delivered_customer_date', 'string'),
 ('order_estimated_delivery_date', 'string'),
 ('payment_sequential', 'int'),
 ('payment_type', 'string'),
 ('payment_installments', 'int'),
 ('payment_value', 'double'),
 ('review_id', 'string'),
 ('review_score', 'string'),
 ('review_comment_title', 'string'),
 ('review_comment_message', 'string'),
 ('review_creation_date', 'string'),
 ('review_answer_timestamp', 'string')]

## 5 - Qual o ticket médio dos pedidos?

In [ ]:
## Code

## 6 - Qual o dia da semana com maior número de pedidos?

In [ ]:
## Code

## 7 - Quais são os Top 10 produtos mais bem avaliados? O produto deve ter pelo menos 5 avaliações.

In [ ]:
## Code

## 8 - Qual o faturamento mensal por método de pagamento?

In [ ]:
## Code

## 9 - Qual a categoria mais vendida na empresa?

In [ ]:
## Code

## 10 - Qual vendedor teve a melhor performance mês a mês?

In [ ]:
## Code